In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import onnxmltools
import onnx
from onnxsim import simplify

In [6]:
model = Sequential(
    [
        Dense(32, activation="relu", input_dim=100, name="main_input"),
        Dense(1, activation="sigmoid", name="dense_1"),
    ]
)


model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

#Save in tf's format, for benchmarking
model.save('my_model')


Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 0.7127 - accuracy: 0.4870
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7004 - accuracy: 0.5130
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5280
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6888 - accuracy: 0.5440
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6846 - accuracy: 0.5540
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6815 - accuracy: 0.5660
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6768 - accuracy: 0.5760
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6750 - accuracy: 0.5850
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6719 - accuracy: 0.6050
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6664 - accuracy: 0.6030


INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [7]:
# Saving it in ONNX format makes it easier to
# pass to Rust
onnx_model = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_model, "example.onnx")


# load your predefined ONNX model
model = onnx.load("example.onnx")

# convert model
model_simp, check = simplify(model)

assert check, "Simplified ONNX model could not be validated"

onnxmltools.utils.save_model(model_simp, "simplified.onnx")

2022-12-17 00:15:28.186749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-17 00:15:28.186805: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-17 00:15:28.186935: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-12-17 00:15:28.187502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-17 00:15:28.187520: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/